In [1]:
from library.objective_function import *
from library.optimiser import *
from library.post_analysis import *
from library.experiments import *
%matplotlib inline

**naming of objective function**
ackley -> ak 

tuned ackley -> tak

bukin -> bk

egg holder -> eh

**naming of optimizer**

original CMA -> no

line search CMA-> ls

round off CMA-> ro

**naming of experiments**

experiments -> exps + objective function + optimizer + [hyperparameter]

In [2]:
np.random.seed(23191)

# init multiple experiments
exps_ak_no = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

# One experiment: setup optimizer
cma = cma_es()
optmizerParas = {'x0': np.zeros((2,)),
         'std': np.ones((2,)) * 3, 
         'tol': 1e-5, 
         'adjust_func': do_nothing(), 
         'record': False, 
         'verbose': False}
cma.set_parameters(optmizerParas)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps_ak_no.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
zoneParas = {'origin': np.array([0,0]),
             'edge': np.array([500, 500]),
             'step': 100,
             'size': 10,
             'sym': True}
exps_ak_no.set_sample_zone(zoneParas)

%time data_exps_ak_no = exps_ak_no.do()

CPU times: user 13.9 s, sys: 101 ms, total: 14 s
Wall time: 14.2 s


In [77]:
num = zoneParas['edge'][0] / zoneParas['step']
num = (num + 1) * num / 2
num

15.0

In [34]:
data_exps_ak_no['cost'].sum() / num

19.821937003313206

In [19]:
data_exps_ak_no['res'].sum() / num

0.02666666666666667

In [22]:
data_exps_ak_no['evals'].sum() / num

1459.48

In [23]:
np.random.seed(23191)

# init multiple experiments
exps_ak_ls = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

# One experiment: setup optimizer
cma = cma_es()
optmizerParas = {'x0': np.zeros((2,)),
         'std': np.ones((2,)) * 3, 
         'tol': 1e-5, 
         'adjust_func': line_search(), 
         'record': False, 
         'verbose': False}
cma.set_parameters(optmizerParas)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps_ak_ls.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
zoneParas = {'origin': np.array([0,0]),
             'edge': np.array([500, 500]),
             'step': 100,
             'size': 10,
             'sym': True}
exps_ak_ls.set_sample_zone(zoneParas)

%time data_ak_ls = exps_ak_ls.do()

/Users/huajian/Non-convex/library/optimiser.py:199: RuntimeWarning:

invalid value encountered in sqrt

/Users/huajian/miniconda3/envs/ada/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning:

invalid value encountered in reduce

/Users/huajian/Non-convex/library/optimiser.py:90: RuntimeWarning:

invalid value encountered in less

/Users/huajian/Non-convex/library/optimiser.py:93: RuntimeWarning:

invalid value encountered in less



diverge, raise LinAlgError!
CPU times: user 2min 59s, sys: 231 ms, total: 3min
Wall time: 3min


In [27]:
data_ak_ls['res'].sum() / num

0.4133333333333333

In [35]:
data_ak_ls['cost'].sum() / num

11.733333588932046

In [29]:
data_ak_ls['evals'].sum() / num

70947.87333333334

In [30]:
np.random.seed(23191)

# init multiple experiments
exps_ak_ro = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

# One experiment: setup optimizer
cma = cma_es()
optmizerParas = {'x0': np.zeros((2,)),
         'std': np.ones((2,)) * 3, 
         'tol': 1e-5, 
         'adjust_func': round_off(), 
         'record': False, 
         'verbose': False}
cma.set_parameters(optmizerParas)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps_ak_ro.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
zoneParas = {'origin': np.array([0,0]),
             'edge': np.array([500, 500]),
             'step': 100,
             'size': 10,
             'sym': True}
exps_ak_ro.set_sample_zone(zoneParas)

%time data_ak_ro = exps_ak_ro.do()

diverge, raise LinAlgError!
CPU times: user 9.01 s, sys: 69.8 ms, total: 9.08 s
Wall time: 9.23 s


In [31]:
data_ak_ro['res'].sum()/num

0.5066666666666666

In [37]:
data_ak_ro['cost'].sum() / num

9.760844008835326

In [32]:
data_ak_ro['evals'].sum()/num

913.0

In [69]:
class adam(optimizer):
    def __init__(self):
        self.alpha = 0.01
        self.beta_1 = 0.9
        self.beta_2 = 0.999
        self.epsilon = 1e-8
        self.max_iter = 10000
        self.tol = 1e-2
        
    def set_parameters(self, paras):
        self.paras = paras
        self.x0 = paras['x0']
        self.alpha = paras['alpha']
        self.beta_1 = paras['beta_1']
        self.beta_2 = paras['beta_2']
        self.epsilon = paras['epsilon']
        self.max_iter = paras['max_iter']
        self.tol = paras['tol']
        self.verbose = True if 'verbose' not in paras.keys() else paras['verbose']
        self.record = True if 'record' not in paras.keys() else paras['record']
        
    def optimise(self, obj):
        m_t = 0 
        v_t = 0 
        eval_cnt = 1
        x = self.x0
        stats = {}
        stats['status'] = None
        while eval_cnt < self.max_iter:					#till it gets converged
            eval_cnt+=1
            g_t = obj.dfunc(x)		#computes the gradient of the stochastic function
            m_t = self.beta_1*m_t + (1-self.beta_1)*g_t	#updates the moving averages of the gradient
            v_t = self.beta_2*v_t + (1-self.beta_2)*(g_t*g_t)	#updates the moving averages of the squared gradient
            m_cap = m_t/(1-(self.beta_1**eval_cnt))		#calculates the bias-corrected estimates
            v_cap = v_t/(1-(self.beta_2**eval_cnt))		#calculates the bias-corrected estimates
            x_prev = x								
            x = x - (self.alpha*m_cap)/(np.sqrt(v_cap)+self.epsilon)	#updates the parameters
            if(np.linalg.norm(x-x_prev) < self.tol):		#checks if it is converged or not
                break
        stats['evals'] = eval_cnt
        return x, obj.func(x), stats

In [70]:
# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

# One experiment: setup optimizer
ad = adam()
optmizerParas = {'x0': np.array([133,23]),
         'alpha': 1e-9,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-8, 
         'max_iter': 1000,
         'tol': 1e-2,              
         'verbose': True,
         'record': True}
ad.set_parameters(optmizerParas)
exp.set_optimizer(ad)
exp.do()


Result:  local minimum
Cost_val 19.999999897405754


In [72]:
for i in range(0, 10):
    # One experiment: setup optimizer
    ad = adam()
    optmizerParas = {'x0': np.array([133,23]),
             'alpha': np.power(0.1, i),
             'beta_1': 0.9, 
             'beta_2': 0.999, 
             'epsilon': 1e-8, 
             'max_iter': 1000,
             'tol': 1e-2,              
             'verbose': True,
             'record': True}
    ad.set_parameters(optmizerParas)
    exp.set_optimizer(ad)
    exp.do()


Result:  local minimum
Cost_val 20.003733434069357

Result:  local minimum
Cost_val 20.02543578963327

Result:  local minimum
Cost_val 20.00040829469172

Result:  local minimum
Cost_val 20.000003981849826

Result:  local minimum
Cost_val 19.999999938249736

Result:  local minimum
Cost_val 19.999999897814142

Result:  local minimum
Cost_val 19.999999897409833

Result:  local minimum
Cost_val 19.999999897405793

Result:  local minimum
Cost_val 19.999999897405754

Result:  local minimum
Cost_val 19.999999897405754


In [71]:
for i in range(0, 10):
    # One experiment: setup optimizer
    ad = adam()
    optmizerParas = {'x0': np.array([133,23]),
             'alpha': np.power(10, i),
             'beta_1': 0.9, 
             'beta_2': 0.999, 
             'epsilon': 1e-8, 
             'max_iter': 1000,
             'tol': 1e-2,              
             'verbose': True,
             'record': True}
    ad.set_parameters(optmizerParas)
    exp.set_optimizer(ad)
    exp.do()


Result:  local minimum
Cost_val 20.003733434069357

Result:  local minimum
Cost_val 20.117767201513793

Result:  local minimum
Cost_val 20.756142458199538

Result:  local minimum
Cost_val 21.957637571413972

Result:  local minimum
Cost_val 21.82917204691484

Result:  local minimum
Cost_val 21.068612259536778

Result:  local minimum
Cost_val 21.75169247669363

Result:  local minimum
Cost_val 21.242426070444008

Result:  local minimum
Cost_val 21.64601113809763

Result:  local minimum
Cost_val 21.391802740896953


In [74]:
np.random.seed(23191)

# init multiple experiments
exps_ak_ad = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

# One experiment: setup optimizer
ad = adam()
optmizerParas = {'x0': np.zeros((2,)),
         'alpha': 1,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-8, 
         'max_iter': 1000,
         'tol': 1e-2,              
         'verbose': False,
         'record': False}
ad.set_parameters(optmizerParas)
exp.set_optimizer(ad)

# Multiple experiment: setup one expreiment
exps_ak_ad.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
zoneParas = {'origin': np.array([0,0]),
             'edge': np.array([500, 500]),
             'step': 100,
             'size': 10,
             'sym': True}
exps_ak_ad.set_sample_zone(zoneParas)

%time data_ak_ad = exps_ak_ad.do()

local minimum 20.0110953961625 55
local minimum 20.514667559280095 14
local minimum 20.000414284557664 95
local minimum 20.003206498782788 67
local minimum 20.006390847617105 63
local minimum 20.011808969011227 55
local minimum 20.009031037873445 66
local minimum 19.99502618859768 83
local minimum 19.99904015779952 79
local minimum 13.476770476754979 40
local minimum 20.002746070969057 86
local minimum 20.074724063754452 54
local minimum 20.081584566788944 46
local minimum 20.273072809866946 42
local minimum 20.00013935774327 94
local minimum 20.22466949620407 24
local minimum 20.00148438802354 77
local minimum 20.00553803319028 64
local minimum 20.00527183813181 58
local minimum 20.006863000115562 65
local minimum 20.322586480390306 32
local minimum 20.003687918962495 98
local minimum 20.002808435367456 79
local minimum 20.006724768665137 59
local minimum 20.019481926026742 44
local minimum 20.003192567193004 72
local minimum 20.00260841416656 73
local minimum 20.001761968823004 85
lo

In [75]:
data_ak_ad

{'x': array([[  0.,   0.,   0.,   0., 400.],
        [  0.,   0.,   0., 300., 400.],
        [  0.,   0., 200., 300., 400.],
        [  0., 100., 200., 300., 400.],
        [  0., 100., 200., 300., 400.]]),
 'y': array([[  0.,   0.,   0.,   0., 400.],
        [  0.,   0.,   0., 300., 300.],
        [  0.,   0., 200., 200., 200.],
        [  0., 100., 100., 100., 100.],
        [  0.,   0.,   0.,   0.,   0.]]),
 'mask': array([[1, 1, 1, 1, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=uint8),
 'res': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 'cost': array([[ 0.        ,  0.        ,  0.        ,  0.        , 20.1896306 ],
        [ 0.        ,  0.        ,  0.        , 20.06084224, 20.03383206],
        [ 0.        ,  0.        , 20.10505559, 20.00885624, 20.22046639],
        [ 0.        , 20.03765323, 20.00708146

In [79]:
data_ak_ad['evals'].sum() / num

62.82

In [80]:
data_ak_ad['res'].sum() / num

0.0

In [81]:
data_ak_ad['cost'].sum() / num

20.021520551849616

## summary on original ackley
| -- |original CMA | line search (default(medium) step size) - CMA  | round off - CMA| adam |
| -- | -- | --| -- | --|
| #experiments| 150 experiments | 150 experiments | 150 experiments | 150 experiments |
| total time| 13.1 s | 180 s |  9.23 s | 0.5 s |
| evalutaions per exp | 1.46 k evals |  7.1 k evals | 0.91 k evals |  0.063 k evals |
| *performance* | -- | -- | -- | --|
| probability |  0.03  | 0.41 | 0.51 | 0 |

In [83]:
np.random.seed(23191)

# init multiple experiments
exps_bk_ad = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
bk = bukin()
exp.set_objective(bk)

# One experiment: setup optimizer
ad = adam()
optmizerParas = {'x0': np.zeros((2,)),
         'alpha': 0.01,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-8, 
         'max_iter': 1000,
         'tol': 1e-2,              
         'verbose': False,
         'record': False}
ad.set_parameters(optmizerParas)
exp.set_optimizer(ad)

# Multiple experiment: setup one expreiment
exps_bk_ad.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
zoneParas = {'origin': np.array([0,0]),
             'edge': np.array([50, 500]),
             'step': 100,
             'size': 10,
             'sym': True}
exps_bk_ad.set_sample_zone(zoneParas)

%time data_bk_ad = exps_bk_ad.do()

local minimum 336.50762505648925 1000
local minimum 43.77392881237146 799
local minimum 38.92956595375648 218
local minimum 48.94529015572177 746
local minimum 914.348430809745 1000
local minimum 898.8253983330126 1000
local minimum 49.14408274752482 984
local minimum 516.1078303169555 1000
local minimum 664.1163445549063 1000
local minimum 16.245675833778737 267
local minimum 1692.9758878048588 1000
local minimum 982.9055766002984 1000
local minimum 710.2947720210384 1000
local minimum 881.0539285853567 1000
local minimum 1439.6617962163755 1000
local minimum 1210.9480059005966 1000
local minimum 54.246030427187534 705
local minimum 1693.3313675947625 1000
local minimum 1050.7919809794603 1000
local minimum 1644.1315234414833 1000
local minimum 30.14033075245077 279
local minimum 922.6489598147454 1000
local minimum 1312.3393800919087 1000
local minimum 1026.5825913956555 1000
local minimum 694.690198950988 1000
local minimum 1166.8370103249028 1000
local minimum 321.9200891063691 100